In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from sklearn.metrics import r2_score
pd.set_option('display.max_columns', None)

In [ ]:
# set up dataframes
# https://www.kaggle.com/datasets/christophercorrea/prisoners-and-crime-in-united-states
JailDeaths = pd.read_csv("/content/drive/MyDrive/datasets/all_deaths.csv", encoding='latin-1')
JailStats = pd.read_csv("/content/drive/MyDrive/datasets/all_jails.csv", encoding='latin-1')

# https://www.kaggle.com/datasets/noriuk/us-education-datasets-unification-project
Education_ByState = pd.read_csv("/content/drive/MyDrive/datasets/states_all.csv", encoding='latin-1')

# https://bjs.ojp.gov/library/publications/prisoners-2021-statistical-tables#:~:text=The%20U.S.%20prison%20population%20was,decrease%20from%202011%20(1%2C599%2C000)
p21stt01 = pd.read_csv("/content/drive/MyDrive/datasets/p21stt01.csv", encoding='latin-1')
p21stat02 = pd.read_csv("/content/drive/MyDrive/datasets/p21stat02.csv", encoding='latin-1')

Education_ByState

In [ ]:
# clean up states_all_extended
# set columns
Education_ByState.columns = [
    "State Key",
    "State",
    "Year",
    "Total Enrollment",
    "Total Revenue",
    "Federal Revenue",
    "State Revenue",
    "Local Revenue",
    "Total Expenditure",
    "Instruction Expenditure",
    "Support Services Expenditure",
    "Other Expenditure",
    "Capital Outlay Expenditure",
    "Total Enrollment (Pre-K)",
    "Total Enrollment (K)",
    "Total Enrollment (4)",
    "Total Enrollment (8)",
    "Total Enrollment (12)",
    "Total Enrollment (1-8)",
    "Total Enrollment (9-12)",
    "Total Enrollment (ALL)",
    "Average Math Score (4)",
    "Average Math Score (8)",
    "Average Reading Score (4)",
    "Average Reading Score (8)",
]

In [ ]:
##Education Dataset Cleanup
Education_ByStateCleaned = Education_ByState.drop(['State Key'], axis = 1)
Education_ByStateCleaned = Education_ByStateCleaned[Education_ByStateCleaned['Year'] <= 2016]
Education_ByStateCleaned = Education_ByStateCleaned[Education_ByStateCleaned['Year'] >= 1992]
Education_ByStateCleaned = Education_ByStateCleaned[Education_ByStateCleaned['State'] != 'NATIONAL']
Education_ByStateCleaned = Education_ByStateCleaned[Education_ByStateCleaned['State'] != 'DODEA']

#Dropping rows that have NA values for Instruction Expenditure
Education_ByStateCleaned = Education_ByStateCleaned.dropna(subset= ['Instruction Expenditure'])


##Filtering Rows that have NAs for both enrollment estimates then filling one rows NAs with the other rows estimates. 
Education_ByStateCleaned[pd.notna(Education_ByStateCleaned['Total Enrollment']) | pd.notna(Education_ByStateCleaned['Total Enrollment (ALL)'])]
Education_ByStateCleaned['Total Enrollment'] = Education_ByStateCleaned['Total Enrollment'].fillna(Education_ByStateCleaned['Total Enrollment (ALL)'])
Education_ByStateCleaned.drop(['Total Enrollment (ALL)'], axis = 1)


#Adding Expenditure Per Student To Clean DF
Education_ByStateCleaned['Expenditure Per Student'] = Education_ByStateCleaned['Instruction Expenditure'] / Education_ByStateCleaned['Total Enrollment']

Education_ByStateCleaned

In [ ]:
##Exploring The Data

##Expenditure Per Student By State
EducationExp = Education_ByStateCleaned

#Selecting appropriate columns
EducationExp = EducationExp[['State', 'Year', 'Expenditure Per Student', 'Average Math Score (4)']]

In [ ]:
#Creating Groupedby DF
EducationExpGroupby = Education_ByStateCleaned.groupby('State').mean()
EducationExpGroupby = EducationExpGroupby.reset_index().sort_values('Expenditure Per Student', ascending= False)
EducationExpGroupby.index = EducationExpGroupby['State'] 

#Graphing Expenditure
fig, ax = plt.subplots(figsize=(8,14))
bars = plt.barh(EducationExpGroupby['State'],EducationExpGroupby['Expenditure Per Student'])
plt.style.use('seaborn-v0_8-notebook')
plt.ylabel("State")
ax.spines[['right','top','bottom']].set_visible(False)
ax.xaxis.set_visible(False)
ax.bar_label(bars)

plt.title("Expenditure Per Student in Each State (1992-2016)")
plt.show()

In [ ]:
#Graphing Math Scores
EducationExpGroupby = EducationExpGroupby.sort_values('Average Math Score (4)', ascending= False)
fig, ax = plt.subplots(figsize=(8,14))
bars = plt.barh(EducationExpGroupby['State'],EducationExpGroupby['Average Math Score (4)'])
plt.style.use('seaborn-v0_8-notebook')
plt.ylabel("State")
ax.spines[['right','top','bottom']].set_visible(False)
ax.xaxis.set_visible(False)
ax.bar_label(bars)

plt.title("Math Scores in Each State (1992-2016)")
plt.show()

In [ ]:
#Exploring Correlations Between Variables
CorrelationEducation = Education_ByStateCleaned

#Create Expenditure By Student
# CorrelationEducation['Expenditure Per Student'] = CorrelationEducation['Instruction Expenditure'] / CorrelationEducation['Total Enrollment']
CorrelationEducation = CorrelationEducation.drop(['State'], axis = 1)
CorrelationEducation = CorrelationEducation[
    pd.notna(CorrelationEducation['Average Math Score (4)']) & pd.notna(CorrelationEducation['Average Math Score (8)'])]


# Scaling data
Scaler = StandardScaler()
Scaler.fit(CorrelationEducation)
CorrelationEducation = pd.DataFrame(Scaler.transform(CorrelationEducation), 
        columns= CorrelationEducation.columns)

#Creating Correlation Matrix
corr = CorrelationEducation.dropna().corr()
plt.figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')
corrMat = plt.matshow(corr, fignum = 1)
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.gca().xaxis.tick_bottom()
plt.colorbar(corrMat)
plt.title(f'Correlation Matrix', fontsize=15)
plt.show()

#Selecting Certain Correlations From The Matrix
CorrelationEducation['Capital Outlay Expenditure'].corr(CorrelationEducation['Average Reading Score (8)'])

In [ ]:
#Creating Linear Regression Models
import math
MSEList = []
ADJRSQUARED = []

#Model1
Model1Data = CorrelationEducation[['Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Year']]
Model1Data = Model1Data[pd.notna(Model1Data['Average Math Score (4)']) & pd.notna(Model1Data['Average Math Score (8)']) 
                                        & pd.notna(Model1Data['Expenditure Per Student']) & pd.notna(Model1Data['Year'])]
TestingFloor = math.floor(len(Model1Data) * .8)
Model1DataTraining = Model1Data[[ 'Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Year']].iloc[0:TestingFloor]
Model1DataTesting = Model1Data[['Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Year']].iloc[TestingFloor:]


model = LinearRegression(fit_intercept= True)
XTraining = Model1DataTraining[['Average Math Score (4)', 'Expenditure Per Student', 'Year']]
YTraining = Model1DataTraining['Average Math Score (8)']
XTesting = Model1DataTesting[['Average Math Score (4)', 'Expenditure Per Student', 'Year']]
YTesting = Model1DataTesting['Average Math Score (8)']
model.fit(XTraining, YTraining)
ModelPredictions = model.predict(XTesting)
MSEList.append(metrics.mean_squared_error(ModelPredictions, YTesting))
ADJRSQUARED.append((1-(1-r2_score(YTesting, ModelPredictions))*((len(XTesting)-1)/(len(XTesting)-len(XTesting.columns)-1))))



#Model2
Model2Data = CorrelationEducation[['Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Year', 'Average Reading Score (4)']]
Model2Data = Model2Data[pd.notna(Model2Data['Average Math Score (4)']) & pd.notna(Model2Data['Average Math Score (8)']) 
                                        & pd.notna(Model2Data['Expenditure Per Student']) & pd.notna(Model2Data['Year'])
                                        & pd.notna(Model2Data['Average Reading Score (4)'])]
TestingFloor = math.floor(len(Model2Data) * .8)
Model2DataTraining = Model2Data[[ 'Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Year', 'Average Reading Score (4)']].iloc[0:TestingFloor]
Model2DataTesting = Model2Data[['Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Year', 'Average Reading Score (4)']].iloc[TestingFloor:]

model = LinearRegression(fit_intercept= True)
XTraining = Model2DataTraining[['Average Math Score (4)', 'Expenditure Per Student', 'Year', 'Average Reading Score (4)']]
YTraining = Model2DataTraining['Average Math Score (8)']
XTesting = Model2DataTesting[['Average Math Score (4)', 'Expenditure Per Student', 'Year', 'Average Reading Score (4)']]
YTesting = Model2DataTesting['Average Math Score (8)']
model.fit(XTraining, YTraining)
ModelPredictions = model.predict(XTesting)
MSEList.append(metrics.mean_squared_error(ModelPredictions, YTesting))
ADJRSQUARED.append((1-(1-r2_score(YTesting, ModelPredictions))*((len(XTesting)-1)/(len(XTesting)-len(XTesting.columns)-1))))
#Model 3 (Final Model)

Model3Data = CorrelationEducation[['Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Average Reading Score (4)', 'Average Reading Score (8)']]
Model3Data = Model3Data[pd.notna(Model3Data['Average Math Score (4)']) & pd.notna(Model3Data['Average Math Score (8)']) 
                                        & pd.notna(Model3Data['Expenditure Per Student']) & pd.notna(Model3Data['Average Reading Score (4)']) & pd.notna(Model3Data['Average Reading Score (8)'])]
Model4DataTraining = Model3Data[[ 'Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Average Reading Score (4)', 'Average Reading Score (8)']].iloc[0:285]
Model4DataTesting = Model3Data[['Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Average Reading Score (4)', 'Average Reading Score (8)']].iloc[285:]

model = LinearRegression(fit_intercept= True)
XTraining = Model4DataTraining[['Average Math Score (4)', 'Expenditure Per Student', 'Average Reading Score (4)', 'Average Reading Score (8)']]
YTraining = Model4DataTraining['Average Math Score (8)']
XTesting = Model4DataTesting[['Average Math Score (4)', 'Expenditure Per Student', 'Average Reading Score (4)', 'Average Reading Score (8)']]
YTesting = Model4DataTesting['Average Math Score (8)']
model.fit(XTraining, YTraining)
ModelPredictions = model.predict(XTesting)
MSEList.append(metrics.mean_squared_error(ModelPredictions, YTesting))
ADJRSQUARED.append((1-(1-r2_score(YTesting, ModelPredictions))*((len(XTesting)-1)/(len(XTesting)-len(XTesting.columns)-1))))
print(ADJRSQUARED)

In [ ]:
plt.scatter(YTesting,ModelPredictions, color='purple', label='Total Enrollment',s=15)

# Create the regression line using the calculated coefficients
regression_line = YTesting 

plt.plot(YTesting, regression_line, color='red', label='Regression Line')

for x, y, ln in zip(YTesting, ModelPredictions, regression_line):
    plt.plot([x, x], [y, ln], color='black', linestyle='--', linewidth=0.5)

plt.show()

In [ ]:
#Polynomial Model
PolynomialModelData = CorrelationEducation[['Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Average Reading Score (4)', 'Average Reading Score (8)']]
PolynomialModelData = PolynomialModelData[pd.notna(PolynomialModelData['Average Math Score (4)']) & pd.notna(PolynomialModelData['Average Math Score (8)']) 
                                        & pd.notna(PolynomialModelData['Expenditure Per Student']) & pd.notna(PolynomialModelData['Average Reading Score (4)']) & pd.notna(PolynomialModelData['Average Reading Score (8)'])]
TestingFloor = math.floor(len(PolynomialModelData) * .8)
PolynomialModelDataTraining = PolynomialModelData[[ 'Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Average Reading Score (4)', 'Average Reading Score (8)']].iloc[0:TestingFloor]
PolynomialModelDataTesting = PolynomialModelData[['Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Average Reading Score (4)', 'Average Reading Score (8)']].iloc[TestingFloor:]

PolyModel1 = make_pipeline(PolynomialFeatures(2, include_bias = False),LinearRegression())
XTraining = PolynomialModelDataTraining[['Average Math Score (4)', 'Expenditure Per Student', 'Average Reading Score (4)', 'Average Reading Score (8)']]
YTraining = PolynomialModelDataTraining['Average Math Score (8)']
PolyModel1.fit(XTraining, YTraining)
XTesting = PolynomialModelDataTesting[['Average Math Score (4)', 'Expenditure Per Student', 'Average Reading Score (4)', 'Average Reading Score (8)']]
YTesting = PolynomialModelDataTesting['Average Math Score (8)']
ModelPredictions = PolyModel1.predict(XTesting)
ADJRSQUARED.append((1-(1-r2_score(YTesting, ModelPredictions))*((len(XTesting)-1)/(len(XTesting)-len(XTesting.columns)-1))))
MSEList.append(metrics.mean_squared_error(ModelPredictions, YTesting))

#Basis 3
PolyModel2 = make_pipeline(PolynomialFeatures(3), LinearRegression())
PolyModel2.fit(XTraining, YTraining)
ModelPredictions = PolyModel2.predict(XTesting)
ADJRSQUARED.append((1-(1-r2_score(YTesting, ModelPredictions))*((len(XTesting)-1)/(len(XTesting)-len(XTesting.columns)-1))))
MSEList.append(metrics.mean_squared_error(ModelPredictions, YTesting))
print(ADJRSQUARED)

In [ ]:
# Exploring why a linear regression outperforms polynomial regression.


# Filter Proper Rows
PMEXPFILTERED = Education_ByStateCleaned[['Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Year', 'Average Reading Score (4)', 'Average Reading Score (8)']]
PMEXPFILTERED = PMEXPFILTERED[pd.notna(PMEXPFILTERED['Average Math Score (4)']) & pd.notna(PMEXPFILTERED['Average Math Score (8)']) 
                                        & pd.notna(PMEXPFILTERED['Expenditure Per Student']) & pd.notna(PMEXPFILTERED['Year'])
                                        & pd.notna(PMEXPFILTERED['Average Reading Score (4)']) & pd.notna(PMEXPFILTERED['Average Reading Score (8)'])]
fig, axs = plt.subplots(2, 2)
plt.sca(axs[0,0])
plt.title('Average Math Score (4)')
plt.sca(axs[0,1])
plt.title('Average Reading Score (4)')
plt.suptitle('Variable Relationships With Average Math Score (8)')

plt.sca(axs[1,0])
plt.title('Expenditure Per Student')
plt.sca(axs[1,1])
plt.title('Average Reading Score (8)')
# Adjust vertical_spacing = 0.5 * axes_height
plt.subplots_adjust(hspace=0.5)



plt.subplot(2,2,1)
plt.scatter(PMEXPFILTERED['Average Math Score (4)'], PMEXPFILTERED['Average Math Score (8)'])

plt.subplot(2,2,2)
plt.scatter(PMEXPFILTERED['Average Reading Score (4)'], PMEXPFILTERED['Average Math Score (8)'])

plt.subplot(2,2,3)
plt.scatter(PMEXPFILTERED['Expenditure Per Student'], PMEXPFILTERED['Average Math Score (8)'])

plt.subplot(2,2,4)
plt.scatter(PMEXPFILTERED['Average Reading Score (8)'], PMEXPFILTERED['Average Math Score (8)'])

In [ ]:
#Neural Network Regression
NNData = CorrelationEducation[['Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Year', 'Average Reading Score (4)', 'Average Reading Score (8)']]
NNData = NNData[pd.notna(NNData['Average Math Score (4)']) & pd.notna(NNData['Average Math Score (8)']) 
                                        & pd.notna(NNData['Expenditure Per Student']) & pd.notna(NNData['Year'])
                                        & pd.notna(NNData['Average Reading Score (4)']) & pd.notna(NNData['Average Reading Score (8)'])]
TestingFloor = math.floor(len(NNData) * .8)
NNDataDataTraining = NNData[[ 'Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Year', 'Average Reading Score (4)', 'Average Reading Score (8)']].iloc[0:TestingFloor]
NNDataDataTesting = NNData[['Average Math Score (4)', 'Expenditure Per Student', 'Average Math Score (8)', 'Year', 'Average Reading Score (4)', 'Average Reading Score (8)']].iloc[TestingFloor:]

XTraining = NNDataDataTraining[['Average Math Score (4)', 'Expenditure Per Student', 'Year', 'Average Reading Score (4)', 'Average Reading Score (8)']]
YTraining = NNDataDataTraining['Average Math Score (8)']

XTesting = NNDataDataTesting[['Average Math Score (4)', 'Expenditure Per Student', 'Year', 'Average Reading Score (4)', 'Average Reading Score (8)']]
YTesting = NNDataDataTesting['Average Math Score (8)']


#Filter Proper Rows
model = models.Sequential()
model.add(layers.Dense(5,activation='relu', 
					input_shape=(5,)))
model.add(layers.Dense(4,activation='relu'))
model.add(layers.Dense(4,activation='relu'))
model.add(layers.Dense(4,activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error'])
model.fit(XTraining,YTraining, epochs = 50)
results = model.evaluate(XTesting, YTesting)


MSEList.append(results[1])
print(MSEList[-1])

In [ ]:
#Graphing MSE Per Model
ModelNumber = ['L1',"L2","L3","P2","P3","NN"]
plt.bar(ModelNumber,MSEList)
plt.title("Comparing Respective Mean Squared Errors")
plt.xlabel("Model Number")
plt.ylabel("MSE")
plt.ylim(0, .25)

In [ ]:
#Graphing Adjusted R Squared Per Model
ModelNumber = ['L1',"L2","L3","P2","P3"]
plt.bar(ModelNumber,ADJRSQUARED)
plt.title("Comparing Respective Adjusted R Squared")
plt.xlabel("Model Number")
plt.ylabel("Adjusted R Squared")
plt.ylim(0, 1)

In [ ]:
#Importing Jail Data
# https://www.kaggle.com/datasets/christophercorrea/prisoners-and-crime-in-united-states?select=crime_and_incarceration_by_state.csv
crimesAndIncarcerationsByState = pd.read_csv("/content/drive/MyDrive/datasets/crime_and_incarceration_by_state.csv", encoding='latin-1')
prisonCustodyByState = pd.read_csv("/content/drive/MyDrive/datasets/prison_custody_by_state.csv", encoding='latin-1')
crimeTotalsByState = pd.read_csv("/content/drive/MyDrive/datasets/ucr_by_state.csv", encoding='latin-1')

In [ ]:
#crimetotal cleanup
crimeTotalsByStateCleaned = crimeTotalsByState.drop(["Unnamed: 15", "Unnamed: 16", "Unnamed: 17", "Unnamed: 18", "Unnamed: 19", "Unnamed: 20", "crime_reporting_change", "crimes_estimated", "rape_legacy", 'rape_revised'], axis=1)
crimeTotalsByStateCleaned = crimeTotalsByStateCleaned.drop(range(869, 948)).reset_index(drop=True)
crimeTotalsByStateCleaned = crimeTotalsByStateCleaned.rename(columns={"ï»¿jurisdiction": "State", "year": "Year"})
crimeTotalsByStateCleaned["Year"] = crimeTotalsByStateCleaned["Year"].astype(int)
crimeTotalsByStateCleaned = crimeTotalsByStateCleaned[crimeTotalsByStateCleaned['Year'] >= 2001]
crimeTotalsByStateCleaned = crimeTotalsByStateCleaned[crimeTotalsByStateCleaned['Year'] <= 2016]

crimeTotalsByStateCleaned['State'] = crimeTotalsByStateCleaned['State'].str.upper()

#Cleaning all the columns with commas and converting to int
crimeTotalsByStateCleaned['state_population'] = crimeTotalsByStateCleaned['state_population'].str.replace(',', '').astype(int)
crimeTotalsByStateCleaned['violent_crime_total'] = crimeTotalsByStateCleaned['violent_crime_total'].str.replace(',', '').astype(int)
crimeTotalsByStateCleaned['murder_manslaughter'] = crimeTotalsByStateCleaned['murder_manslaughter'].str.replace(',', '').astype(int)
# crimeTotalsByStateCleaned['rape_legacy'] = crimeTotalsByStateCleaned['rape_legacy'].str.replace(',', '').astype(int)
# crimeTotalsByStateCleaned['rape_revised'] = crimeTotalsByStateCleaned['rape_revised'].str.replace(',', '').astype(int)
crimeTotalsByStateCleaned['robbery'] = crimeTotalsByStateCleaned['robbery'].str.replace(',', '').astype(int)
crimeTotalsByStateCleaned['agg_assault'] = crimeTotalsByStateCleaned['agg_assault'].str.replace(',', '').astype(int)
crimeTotalsByStateCleaned['property_crime_total'] = crimeTotalsByStateCleaned['property_crime_total'].str.replace(',', '').astype(int)
crimeTotalsByStateCleaned['burglary'] = crimeTotalsByStateCleaned['burglary'].str.replace(',', '').astype(int)
crimeTotalsByStateCleaned['larceny'] = crimeTotalsByStateCleaned['larceny'].str.replace(',', '').astype(int)
crimeTotalsByStateCleaned['vehicle_theft'] = crimeTotalsByStateCleaned['vehicle_theft'].str.replace(',', '').astype(int)

crimeTotalsByStateCleaned = crimeTotalsByStateCleaned.rename(columns={"state_population": "State Population", 
                                                                      "violent_crime_total": "Violent Total Crime",
                                                                      "murder_manslaughter": "Murder",
                                                                      "rape_revised": "Rape(Revised)",
                                                                      "robbery": "Robbery",
                                                                      "agg_assault": "Agg Assault",
                                                                      "property_crime_total": "Property Crime Total",
                                                                      "burglary": "Burglary",
                                                                      "larceny": "Larceny",
                                                                      "vehicle_theft": "Vehicle Theft"})

In [ ]:
#Merging DFs
CrimeEducationMerged = pd.merge(Education_ByStateCleaned, crimeTotalsByStateCleaned, on=['State', 'Year'])

pd.set_option('display.max_columns', None)
display(CrimeEducationMerged)

In [ ]:
##Exploring The Jail Data

##Support Expenditure Per Student By State
EducationSupportExp = CrimeEducationMerged

#Selecting appropriate columns
EducationSupportExp = EducationSupportExp[['State', 'Year', 'Total Enrollment', 'Support Services Expenditure','Vehicle Theft']]

#Create Expenditure By Student
EducationSupportExp['Support Per Student'] = EducationSupportExp['Support Services Expenditure'] / EducationSupportExp['Total Enrollment']
EducationSupportExpGroupby = EducationSupportExp.groupby('State').mean()
EducationSupportExpGroupby = EducationSupportExpGroupby.reset_index().sort_values('Support Per Student', ascending= False)

EducationSupportExpGroupby.index = EducationSupportExpGroupby['State'] 

In [ ]:
#Creating Groupedby DF
EducationSupportExpGroupby = EducationSupportExp.groupby('State').mean()
EducationSupportExpGroupby = EducationSupportExpGroupby.reset_index().sort_values('Support Per Student', ascending= False)
EducationSupportExpGroupby.index = EducationSupportExpGroupby['State'] 

#Graphing Support
fig, ax = plt.subplots(figsize=(8,14))
bars = plt.barh(EducationSupportExpGroupby['State'],EducationSupportExpGroupby['Support Per Student'])
plt.style.use('seaborn-v0_8-notebook')
plt.ylabel("State")
ax.spines[['right','top','bottom']].set_visible(False)
ax.xaxis.set_visible(False)
ax.bar_label(bars)

plt.title("Support Per Student in Each State (2001-2016)")
plt.show()

In [ ]:
EducationSupportExpGroupby = EducationSupportExp.groupby('State').mean()
EducationSupportExpGroupby['Theft Per Person'] = EducationSupportExpGroupby['Vehicle Theft']/EducationSupportExpGroupby['Total Enrollment']
EducationSupportExpGroupby = EducationSupportExpGroupby.reset_index().sort_values('Theft Per Person', ascending= False)
fig, ax = plt.subplots(figsize=(6,10))
bars = plt.barh(EducationSupportExpGroupby['State'],EducationSupportExpGroupby['Theft Per Person'])
plt.style.use('seaborn-v0_8-notebook')
plt.ylabel("State")
ax.spines[['right','top','bottom']].set_visible(False)
ax.xaxis.set_visible(False)
ax.bar_label(bars)

plt.title("Standardized Vehicle Thefts in Each State (2001-2016)")
plt.show()

In [ ]:
#Exploring Correlations Between Variables
CorrelationCrime = CrimeEducationMerged

#Create Expenditure By Student
CorrelationCrime = CorrelationCrime.drop(['State'], axis = 1)
# CorrelationCrime = CorrelationCrime[pd.notna(CorrelationEducation['Average Math Score (4)']) & pd.notna(CorrelationEducation['Average Math Score (8)'])]


# Scaling data
Scaler = StandardScaler()
Scaler.fit(CorrelationCrime)
CorrelationCrime = pd.DataFrame(Scaler.transform(CorrelationCrime), 
        columns= CorrelationCrime.columns)

#Creating Correlation Matrix
corr = CorrelationCrime.dropna().corr()
plt.figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')
corrMat = plt.matshow(corr, fignum = 1)
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.gca().xaxis.tick_bottom()
plt.colorbar(corrMat)
plt.title(f'Correlation Matrix', fontsize=15)
plt.show()

#Selecting Certain Correlations From The Matrix
CorrelationCrime['Support Services Expenditure'].corr(CorrelationCrime['Vehicle Theft'])

In [ ]:
#Predicting Vehicle Theft

#Creating ADJ R2 List
ADJRSQUAREDJAIL= []

#MSE List
MSEListJail = []

#Model 1
Model1Data = CrimeEducationMerged[['Violent Total Crime', 'Support Services Expenditure', 'Vehicle Theft']]
Model1Data = Model1Data[pd.notna(Model1Data['Violent Total Crime']) & pd.notna(Model1Data['Support Services Expenditure']) & 
                        pd.notna(Model1Data['Vehicle Theft'])]

TestingFloor = math.floor(len(Model1Data) * .8)
Model1DataTraining = Model1Data.iloc[0:TestingFloor]
Model1DataTesting = Model1Data.iloc[TestingFloor:]


model = LinearRegression(fit_intercept= True)
XTraining = Model1DataTraining[['Violent Total Crime', 'Support Services Expenditure']]
YTraining = Model1DataTraining['Vehicle Theft']
XTesting = Model1DataTesting[['Violent Total Crime', 'Support Services Expenditure']]

print(XTesting)
print(len(XTesting))

YTesting = Model1DataTesting['Vehicle Theft']
model.fit(XTraining, YTraining)
ModelPredictions = model.predict(XTesting)
MSEListJail.append(metrics.mean_squared_error(ModelPredictions, YTesting))
ADJRSQUAREDJAIL.append((1-(1-r2_score(YTesting, ModelPredictions))*((len(XTesting)-1)/(len(XTesting)-len(XTesting.columns)-1))))

#Model2 
Model2Data = CrimeEducationMerged[['Violent Total Crime', 'Support Services Expenditure', 'Vehicle Theft', 'State Revenue']]
Model2Data = Model2Data[pd.notna(Model2Data['Violent Total Crime']) & pd.notna(Model2Data['Support Services Expenditure']) & 
                        pd.notna(Model2Data['Vehicle Theft']) & pd.notna(Model2Data['State Revenue'])]

TestingFloor = math.floor(len(Model2Data) * .8)
Model2DataTraining = Model2Data.iloc[0:TestingFloor]
Model2DataTesting = Model2Data.iloc[TestingFloor:]


model = LinearRegression(fit_intercept= True)
XTraining = Model2DataTraining[['Violent Total Crime', 'Support Services Expenditure', 'State Revenue']]
YTraining = Model2DataTraining['Vehicle Theft']
XTesting = Model2DataTesting[['Violent Total Crime', 'Support Services Expenditure', 'State Revenue']]
YTesting = Model2DataTesting['Vehicle Theft']
model.fit(XTraining, YTraining)
ModelPredictions = model.predict(XTesting)
MSEListJail.append(metrics.mean_squared_error(ModelPredictions, YTesting))
ADJRSQUAREDJAIL.append((1-(1-r2_score(YTesting, ModelPredictions))*((len(XTesting)-1)/(len(XTesting)-len(XTesting.columns)-1))))

#Polynomial Model
PolyModel = CrimeEducationMerged[['Violent Total Crime', 'Support Services Expenditure', 'Vehicle Theft', 'State Revenue']]
PolyModel = PolyModel[pd.notna(PolyModel['Violent Total Crime']) & pd.notna(PolyModel['Support Services Expenditure']) & 
                        pd.notna(PolyModel['Vehicle Theft']) & pd.notna(PolyModel['State Revenue'])]

TestingFloor = math.floor(len(PolyModel) * .8)
PolyModelTraining = PolyModel.iloc[0:TestingFloor]
PolyModelTesting = PolyModel.iloc[TestingFloor:]


model = make_pipeline(PolynomialFeatures(2), LinearRegression(fit_intercept= True))
XTraining = PolyModelTraining[['Violent Total Crime', 'Support Services Expenditure', 'State Revenue']]
YTraining = PolyModelTraining['Vehicle Theft']
XTesting = PolyModelTesting[['Violent Total Crime', 'Support Services Expenditure', 'State Revenue']]
YTesting = PolyModelTesting['Vehicle Theft']
model.fit(XTraining, YTraining)
ModelPredictions = model.predict(XTesting)
MSEListJail.append(metrics.mean_squared_error(ModelPredictions, YTesting))
ADJRSQUAREDJAIL.append((1-(1-r2_score(YTesting, ModelPredictions))*((len(XTesting)-1)/(len(XTesting)-len(XTesting.columns)-1))))

#EducationModel
Model2Data = CrimeEducationMerged[['Support Services Expenditure', 'Vehicle Theft', 'State Revenue', 'Total Enrollment']]
Model2Data = Model2Data[pd.notna(Model2Data['Total Enrollment']) & pd.notna(Model2Data['Support Services Expenditure']) & 
                        pd.notna(Model2Data['Vehicle Theft']) & pd.notna(Model2Data['State Revenue'])]

TestingFloor = math.floor(len(Model2Data) * .8)
Model2DataTraining = Model2Data.iloc[0:TestingFloor]
Model2DataTesting = Model2Data.iloc[TestingFloor:]


model = LinearRegression(fit_intercept= True)
XTraining = Model2DataTraining[['Total Enrollment', 'Support Services Expenditure', 'State Revenue']]
YTraining = Model2DataTraining['Vehicle Theft']
XTesting = Model2DataTesting[['Total Enrollment', 'Support Services Expenditure', 'State Revenue']]
YTesting = Model2DataTesting['Vehicle Theft']
model.fit(XTraining, YTraining)
ModelPredictions = model.predict(XTesting)
print(metrics.r2_score(ModelPredictions, YTesting))
MSEListJail.append(metrics.mean_squared_error(ModelPredictions, YTesting))
ADJRSQUAREDJAIL.append((1-(1-r2_score(YTesting, ModelPredictions))*((len(XTesting)-1)/(len(XTesting)-len(XTesting.columns)-1))))


In [ ]:
plt.scatter(YTesting,ModelPredictions, color='purple', label='Total Enrollment',s=15)

# Create the regression line using the calculated coefficients
regression_line = YTesting 

plt.plot(YTesting, regression_line, color='red', label='Regression Line')

for x, y, ln in zip(YTesting, ModelPredictions, regression_line):
    plt.plot([x, x], [y, ln], color='black', linestyle='--', linewidth=0.5)

plt.show()

In [ ]:
ModelNumber = ["1","2","3","4"]
plt.grid(True, zorder = 0, color = "grey", linewidth = "1", linestyle = "--")
plt.bar(ModelNumber,ADJRSQUAREDJAIL,zorder=3)
plt.title("Adj R^2 By Model")
plt.xlabel("Model Number")
plt.ylabel("Adj R^2 Value")
plt.show()

In [ ]:
ModelNumber = ["1","2","3","4"]
plt.grid(True, zorder = 0, color = "grey", linewidth = "1", linestyle = "--")
plt.bar(ModelNumber,MSEListJail,zorder=3)
plt.title("MSE By Model")
plt.xlabel("Model Number")
plt.ylabel("MSE Value")
plt.show()